In [26]:
import numpy as np
from random import shuffle

In [27]:
class Deck(object):
    def __init__(self):
        self.cards = []
    def add(self, card):
        self.cards.append(card)
#     def draw(self, hand):
#         hand.append(self.cards.pop(0))
    def randomize(self):
        shuffle(self.cards)

In [28]:
class Hand(object):
    def __init__(self, deck, num_cards=0):
        self.cards = []
        self.draw(deck, num_cards)
        
    def __getitem__(self, key):
        return self.cards[key]
    
    def __str__(self):
        return str(self.cards)
    
    def __iter__(self):
        for card in self.cards:
            yield card
    
    def draw(self, deck, num_cards=1):
        for _ in range(num_cards):
            self.cards.append(deck.cards.pop(0))
    
    @property
    def num_lands(self):
        nland = 0
        for card in self.cards:
            if isinstance(card, Land):
                nland += 1
        return nland
    
    @property
    def num_cards(self):
        return len(self.cards)


In [29]:
class Card(object):
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return self.name
    def __str__(self):
        return self.name

class Land(Card):
    lands = {"plains":"w", "island":"u", "swamp":"b", "mountain":"r", "forest":"g"}
    def __init__(self, name, colors=None, tapland=False):
        super().__init__(name)
        self.tapland = tapland
        
        if colors is None and self.name in Land.lands:
            colors = Land.lands[self.name]
            
        if not isinstance(colors, list):
            colors = list(colors)
        self.colors = colors
        
    @property
    def white(self):
        return "w" in self.colors
    @property
    def blue(self):
        return "u" in self.colors
    @property
    def black(self):
        return "b" in self.colors
    @property
    def red(self):
        return "r" in self.colors
    @property
    def green(self):
        return "g" in self.colors

In [53]:
class Player(object):
    def __init__(self, deck):
        self.deck = deck
        self.deck.randomize()
        self.hand = Hand(self.deck, 7)
        
    def restart(self):
        self.deck.cards += self.hand.cards
        self.deck.randomize()
        self.hand = Hand(self.deck, 7)
        
    @classmethod
    def create_decklist(cls, num_A, num_B, num_mc):
        deck = Deck()
        for _ in range(num_A):
            deck.add(Land("swamp"))
        for _ in range(num_B):
            deck.add(Land("forest"))
        for _ in range(num_mc):
            deck.add(Land("multicolor", colors=["b", "g"], tapland=True))
        for _ in range(40-len(deck.cards)):
            deck.add(Card("arbitrary"))
        # deck.randomize()
        return cls(deck)
    
    @property
    def cards_in_hand(self):
        return self.hand.num_cards
    
    def draw(self, num_cards=1):
        self.hand.draw(self.deck, num_cards)
            
    def draw_until_n_lands(self, n):
        while self.hand.num_lands < n:
            self.draw(1)
    
    def can_cast_AABB(self):
        AB = [0, 0]
        num_mc = 0
        for i,card in enumerate(self.hand):
            if not isinstance(card, Land):
                continue

            if i==self.cards_in_hand-1 and card.tapland:
                break

            if len(card.colors) > 1:
                num_mc += 1

            if card.green:
                AB[0] += 1
            if card.black:
                AB[1] += 1

        for _ in range(num_mc):
            AB = sorted(AB)
            AB[1] -= 1

        result = AB[0] >= 2 and AB[1] >= 2
        return result


In [57]:
def single_run(p, play=True, max_turns=9):
    p.restart()
    p.draw(4-int(play))
    success = []
    for _ in range(4, max_turns):
        success.append(p.can_cast_AABB())
        p.draw()
    return success

def AABB_mc(p, play=True, max_turns=9, num_mc=10000):
    result = [single_run(p, max_turns=max_turns, play=play) for _ in range(num_mc)]
    prob = np.mean(result, axis=0)
    return prob

In [58]:
def full_simulation(num_lands=17, play=True, num_mc=10000):
    if play: play_str="play"
    else: play_str="draw"
    print(f"Simulating {num_lands} lands on the {play_str}")
    for num_mclr in range(6):
        num_swamps = (num_lands - num_mclr)//2
        num_forests = num_lands - num_mclr - num_swamps
        print(num_forests, num_swamps, num_mclr, end="\t")
        p = Player.create_decklist(num_forests, num_swamps, num_mclr)
        prob = AABB_mc(p, play=play, num_mc=num_mc)*100
        prob_str = [f"{x:.1f}" for x in prob]
        print(*prob_str)

In [59]:
full_simulation(17, True, num_mc=10000)
full_simulation(17, False, num_mc=10000)
full_simulation(18, True, num_mc=10000)
full_simulation(18, False, num_mc=10000)

Simulating 17 lands on the play
9 8 0	45.5 54.6 63.2 70.7 77.3
8 8 1	49.7 59.7 68.1 75.1 81.4
8 7 2	52.2 61.6 69.9 77.3 83.4
7 7 3	55.5 65.1 73.6 80.8 86.3
7 6 4	57.1 67.1 75.8 82.5 87.5
6 6 5	60.0 70.0 78.0 85.1 90.2
Simulating 17 lands on the draw
9 8 0	55.0 63.4 70.6 77.5 82.7
8 8 1	59.6 67.7 75.2 81.1 86.1
8 7 2	62.3 70.7 77.9 84.0 88.2
7 7 3	65.7 74.1 81.5 86.7 90.4
7 6 4	66.7 75.6 82.6 88.0 91.6
6 6 5	69.4 77.8 84.9 89.7 93.2
Simulating 18 lands on the play
9 9 0	51.9 61.1 69.1 76.3 82.3
9 8 1	54.4 64.2 72.3 78.7 84.7
8 8 2	57.3 66.7 74.5 81.5 86.6
8 7 3	60.8 70.1 78.2 84.6 89.3
7 7 4	63.4 73.2 81.1 87.1 91.2
7 6 5	64.9 74.5 82.3 88.2 92.1
Simulating 18 lands on the draw
9 9 0	60.3 68.9 76.2 82.1 86.9
9 8 1	63.9 72.4 79.1 84.4 88.4
8 8 2	67.9 76.0 82.5 87.4 91.2
8 7 3	70.4 78.7 84.7 89.4 92.8
7 7 4	71.7 79.5 85.4 90.1 93.8
7 6 5	75.2 82.3 87.7 91.9 95.0


In [60]:
def minimum_castable_turn(player, play=True):
    player.restart()
    turn = 4
    player.draw(turn-int(play))
    while not player.can_cast_AABB():
        player.draw()
        turn += 1
    return turn

In [61]:
def min_turn_simulation(num_lands=17, play=True, num_mc=10000, max_mclr=6):
    if play: play_str="play"
    else: play_str="draw"
    print(f"Simulating {num_lands} lands on the {play_str}")
    for num_mclr in range(max_mclr):
        num_swamps = (num_lands - num_mclr)//2
        num_forests = num_lands - num_mclr - num_swamps
        print(num_forests, num_swamps, num_mclr, end="\t")
        p = Player.create_decklist(num_forests, num_swamps, num_mclr)
        turn = [minimum_castable_turn(p, play) for _ in range(num_mc)]
        print("{:.2f}".format(np.mean(turn)))

In [62]:
min_turn_simulation(17, True, 10000, 6)
min_turn_simulation(17, False, 10000, 6)
min_turn_simulation(18, True, 10000, 6)
min_turn_simulation(18, False, 10000, 6)

Simulating 17 lands on the play
9 8 0	6.49
8 8 1	6.13
8 7 2	5.88
7 7 3	5.66
7 6 4	5.50
6 6 5	5.32
Simulating 17 lands on the draw
9 8 0	5.90
8 8 1	5.60
8 7 2	5.42
7 7 3	5.22
7 6 4	5.08
6 6 5	4.95
Simulating 18 lands on the play
9 9 0	6.06
9 8 1	5.83
8 8 2	5.53
8 7 3	5.42
7 7 4	5.24
7 6 5	5.07
Simulating 18 lands on the draw
9 9 0	5.57
9 8 1	5.32
8 8 2	5.11
8 7 3	4.97
7 7 4	4.83
7 6 5	4.75


In [52]:
p = Player.create_decklist(6,6,5)
p.draw_until_n_lands(4)
print(p.hand)
print(p.can_cast_AABB())

[multicolor, arbitrary, arbitrary, swamp, arbitrary, arbitrary, arbitrary, multicolor, arbitrary, arbitrary, arbitrary, arbitrary, multicolor]
True
